In [ ]:
# ========================================
# EVALUATION FRAMEWORK - SETUP CELL
# Run this first in every notebook
# ========================================

# 1. Mount Google Drive (for data persistence)
from google.colab import drive
drive.mount('/content/drive')

# 2. Set working directory
import os
WORK_DIR = '/content/drive/MyDrive/data_preparation_project_2026'
os.makedirs(WORK_DIR, exist_ok=True)
os.chdir(WORK_DIR)

# Create folder structure
os.makedirs('data/baseline', exist_ok=True)
os.makedirs('data/corrupted', exist_ok=True)
os.makedirs('data/cleaned', exist_ok=True)
os.makedirs('results', exist_ok=True)
os.makedirs('figures', exist_ok=True)

print(f"✅ Working directory: {WORK_DIR}")
print(f"✅ Folder structure created")

# 3. Install required packages
# !pip install -q tensorflow-data-validation  # For Part 2
!pip install -q statsmodels  # For Part 3 (McNemar's test)

# 4. Import common libraries
# import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from scipy.stats import ttest_rel, ks_2samp
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ All libraries loaded")
print("=" * 50)

In [ ]:
!pip install datasets scikit-learn pandas numpy setuptools -q

In [ ]:
!pip install jenga

In [ ]:
beauty_path = "/content/drive/MyDrive/data_preparation_project_2026/data/raw/All_Beauty.jsonl.gz"

import os
print("exists:", os.path.exists(beauty_path))
print("size (MB):", os.path.getsize(beauty_path) / (1024*1024))


In [ ]:
import gzip

with gzip.open(beauty_path, "rt", encoding="utf-8", errors="replace") as f:
    for i in range(3):
        print("LINE", i, ":", f.readline()[:300])


In [ ]:
import gzip

with gzip.open(beauty_path, "rt", encoding="utf-8") as f:
    for _ in range(5):
        print(f.readline())

In [ ]:
import pandas as pd
beauty_df = pd.read_json(beauty_path, lines=True, compression="gzip")
print(len(beauty_df))
beauty_df.head()

In [ ]:
!ls

!pwd

!cd /content/drive/MyDrive/data_preparation_project_2026

### Part 1: Baseline Metrics Comparison

In [ ]:
"""
Quick script to check corrupted file sizes and clean bad checkpoints
"""
import os
import pandas as pd

BASE_DIR = "/content/drive/MyDrive/data_preparation_project_2026"
FIXED_DIR = os.path.join(BASE_DIR, "fixed")

print("="*60)
print("CHECKING CORRUPTED FILES")
print("="*60)

corrupted_dir = os.path.join(FIXED_DIR, "corrupted")
if os.path.exists(corrupted_dir):
    files = sorted([f for f in os.listdir(corrupted_dir) if f.endswith('.csv')])
    for f in files:
        path = os.path.join(corrupted_dir, f)
        df = pd.read_csv(path)
        print(f"{f:40s} → {len(df):,} rows")
else:
    print("❌ No corrupted directory found")

print("\n" + "="*60)
print("DELETING ALL CHECKPOINTS (force re-run)")
print("="*60)

checkpoint_dir = os.path.join(FIXED_DIR, "checkpoints")
if os.path.exists(checkpoint_dir):
    files = [f for f in os.listdir(checkpoint_dir) if f.endswith('.json')]
    for f in files:
        path = os.path.join(checkpoint_dir, f)
        os.remove(path)
        print(f"✅ Deleted: {f}")
    print(f"\n✅ Deleted {len(files)} checkpoints")
else:
    print("❌ No checkpoint directory found")

print("\n" + "="*60)
print("CHECKING BASELINE")
print("="*60)

baseline_path = os.path.join(FIXED_DIR, "baseline/amazon_clean_with_rowid.csv")
if os.path.exists(baseline_path):
    df = pd.read_csv(baseline_path)
    print(f"Baseline with row_id: {len(df):,} rows")
    print(f"Columns: {list(df.columns)}")
else:
    print("❌ No baseline found")

print("\n✅ Ready to re-run evaluation pipeline")

In [ ]:
"""
BATCH 1 EVALUATION - First 4 Corruptions
Runs: missing_values, broken_chars, swapped_text, missing_labels
"""

import os, sys
import pandas as pd
import numpy as np
from datetime import datetime
import gc
import json

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIGURATION
# =========================
BASE_DIR = "/content/drive/MyDrive/data_preparation_project_2026"
FIXED_DIR = os.path.join(BASE_DIR, "fixed_batch1")
TFIDF_MAX_FEATURES = 5000
RANDOM_STATE = 42
TEST_SIZE = 0.20
SUBSET_SIZE = 100000

# =========================
# IMPORT TEAM CODE
# =========================
sys.path.append(os.path.join(BASE_DIR, "team_code"))
import inject_extreme as inject
import clean

# =========================
# UTILITIES
# =========================
def clear_memory():
    gc.collect()

def ensure_dirs():
    for sub in ["baseline", "corrupted", "cleaned", "results", "splits"]:
        os.makedirs(os.path.join(FIXED_DIR, sub), exist_ok=True)

def prepare_baseline_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["text"] = df["text"].astype(str)
    df["label"] = pd.to_numeric(df["label"], errors="coerce")
    df = df.dropna(subset=["label", "text"])
    df["label"] = df["label"].astype(int)
    df = df[df["text"] != "nan"]
    df = df[df["text"].str.len() > 0]
    df = df.reset_index(drop=True)
    df["row_id"] = df.index
    return df

def valid_row_mask(df: pd.DataFrame) -> pd.Series:
    text = df["text"].astype(str)
    label = pd.to_numeric(df["label"], errors="coerce")
    mask = label.notna() & text.notna()
    mask &= (text != "nan") & (text.str.len() > 0)
    mask &= label.apply(lambda x: float(x).is_integer())
    return mask

def build_model():
    return Pipeline([
        ("tfidf", TfidfVectorizer(max_features=TFIDF_MAX_FEATURES, stop_words="english")),
        ("clf", LogisticRegression(max_iter=1000))
    ])

def compute_metrics(y_true, y_pred):
    return {
        "accuracy": float(accuracy_score(y_true, y_pred)),
        "precision": float(precision_score(y_true, y_pred, average="weighted", zero_division=0)),
        "recall": float(recall_score(y_true, y_pred, average="weighted", zero_division=0)),
        "f1": float(f1_score(y_true, y_pred, average="weighted", zero_division=0)),
    }

def train_and_evaluate(df_train, df_test, allow_invalid=False):
    if allow_invalid:
        train_valid = df_train.copy()
        test_valid = df_test.copy()
        train_valid['text'] = train_valid['text'].fillna('')
        test_valid['text'] = test_valid['text'].fillna('')
        train_valid['label'] = pd.to_numeric(train_valid['label'], errors='coerce').fillna(3).astype(int)
        test_valid['label'] = pd.to_numeric(test_valid['label'], errors='coerce').fillna(3).astype(int)
    else:
        train_mask = valid_row_mask(df_train)
        test_mask = valid_row_mask(df_test)
        train_valid = df_train[train_mask].copy()
        test_valid = df_test[test_mask].copy()

    X_train = train_valid["text"].astype(str).values
    y_train = pd.to_numeric(train_valid["label"], errors="coerce").astype(int).values
    X_test = test_valid["text"].astype(str).values
    y_test = pd.to_numeric(test_valid["label"], errors="coerce").astype(int).values

    model = build_model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    metrics = compute_metrics(y_test, y_pred)
    stats = {
        "train_total": len(df_train),
        "train_valid": len(train_valid),
        "test_total": len(df_test),
        "test_valid": len(test_valid),
    }

    del model, X_train, y_train, X_test, y_test, train_valid, test_valid
    clear_memory()
    return metrics, stats

# =========================
# MAIN
# =========================
print("="*80)
print("BATCH 1 EVALUATION - First 4 Corruptions")
print("="*80)

ensure_dirs()

# Load baseline
print("\n📊 Loading baseline...")
baseline_path = os.path.join(BASE_DIR, "data/baseline/amazon_reviews_1M.csv")
df_raw = pd.read_csv(baseline_path)
print(f"   Loaded: {len(df_raw):,} rows")

if len(df_raw) > SUBSET_SIZE:
    df_raw = df_raw.sample(n=SUBSET_SIZE, random_state=RANDOM_STATE)
    print(f"   Sampled: {SUBSET_SIZE:,} rows")

df_baseline = prepare_baseline_data(df_raw)
del df_raw
clear_memory()
print(f"   Cleaned: {len(df_baseline):,} rows")

# Split
df_train_clean, df_test_clean = train_test_split(
    df_baseline, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=df_baseline["label"]
)
train_ids = set(df_train_clean["row_id"].tolist())
test_ids = set(df_test_clean["row_id"].tolist())
print(f"   Split: {len(df_train_clean):,} train / {len(df_test_clean):,} test")

# Baseline eval
print("\n📈 Baseline evaluation...")
metrics_base, stats_base = train_and_evaluate(df_train_clean, df_test_clean, allow_invalid=False)
print(f"   Acc: {metrics_base['accuracy']:.4f} | Prec: {metrics_base['precision']:.4f} | Rec: {metrics_base['recall']:.4f} | F1: {metrics_base['f1']:.4f}")

all_results = []
def record_result(exp_id, stage, metrics, stats):
    row = {"experiment": exp_id, "stage": stage, **stats, **metrics}
    all_results.append(row)

record_result("BASELINE", "CLEAN", metrics_base, stats_base)

# Experiments
experiments = [
    ("01_missing_values", inject.apply_missing_values, {}),
    ("02_broken_chars", inject.apply_broken_characters, {}),
    ("03_swapped_text", inject.apply_swapped_text, {}),
    ("04_missing_labels", inject.apply_missing_labels, {}),
]

for idx, (exp_id, corrupt_func, kwargs) in enumerate(experiments, start=1):
    print(f"\n[{idx}/4] {exp_id}")
    print("-" * 80)

    # Corrupt
    print("   🔴 Corrupting...")
    df_corrupt = corrupt_func(df_baseline.copy(), **kwargs)
    df_corrupt.to_csv(os.path.join(FIXED_DIR, f"corrupted/{exp_id}.csv"), index=False)

    # Split
    df_train_corrupt = df_corrupt[df_corrupt["row_id"].isin(train_ids)].copy()
    df_test_corrupt = df_corrupt[df_corrupt["row_id"].isin(test_ids)].copy()

    # Eval corrupted
    print("   🔴 Training on CORRUPTED...")
    metrics_corrupt, stats_corrupt = train_and_evaluate(df_train_corrupt, df_test_corrupt, allow_invalid=True)
    record_result(exp_id, "CORRUPTED", metrics_corrupt, stats_corrupt)
    print(f"      Acc: {metrics_corrupt['accuracy']:.4f} | Prec: {metrics_corrupt['precision']:.4f} | Rec: {metrics_corrupt['recall']:.4f} | F1: {metrics_corrupt['f1']:.4f}")

    del df_train_corrupt, df_test_corrupt
    clear_memory()

    # Clean
    print("   🟢 Cleaning...")
    df_cleaned = clean.clean_all(df_corrupt)
    df_cleaned.to_csv(os.path.join(FIXED_DIR, f"cleaned/{exp_id}_cleaned.csv"), index=False)
    del df_corrupt
    clear_memory()

    # Split
    df_train_cleaned = df_cleaned[df_cleaned["row_id"].isin(train_ids)].copy()
    df_test_cleaned = df_cleaned[df_cleaned["row_id"].isin(test_ids)].copy()

    # Eval cleaned
    print("   🟢 Training on CLEANED...")
    metrics_cleaned, stats_cleaned = train_and_evaluate(df_train_cleaned, df_test_cleaned, allow_invalid=False)
    record_result(exp_id, "CLEANED", metrics_cleaned, stats_cleaned)
    print(f"      Acc: {metrics_cleaned['accuracy']:.4f} | Prec: {metrics_cleaned['precision']:.4f} | Rec: {metrics_cleaned['recall']:.4f} | F1: {metrics_cleaned['f1']:.4f}")

    # Recovery
    acc_drop = metrics_base['accuracy'] - metrics_corrupt['accuracy']
    acc_recovery = metrics_cleaned['accuracy'] - metrics_corrupt['accuracy']
    recovery_pct = (acc_recovery / acc_drop * 100) if acc_drop != 0 else 0
    print(f"      📊 Recovery: {recovery_pct:.1f}%")

    del df_train_cleaned, df_test_cleaned, df_cleaned
    clear_memory()

# Save results
results_path = os.path.join(FIXED_DIR, "results/batch1_results.csv")
pd.DataFrame(all_results).to_csv(results_path, index=False)

print("\n" + "="*80)
print("BATCH 1 COMPLETE!")
print(f"✅ Results: {results_path}")
print("="*80)

In [ ]:
"""
BATCH 2 EVALUATION - Remaining 6 Corruptions
Runs: swapped_labels_manual, noise_injection, truncation, combined_jenga, combined_manual, doomsday
"""

import os, sys
import pandas as pd
import numpy as np
from datetime import datetime
import gc
import json

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIGURATION
# =========================
BASE_DIR = "/content/drive/MyDrive/data_preparation_project_2026"
FIXED_DIR = os.path.join(BASE_DIR, "fixed_batch2")
TFIDF_MAX_FEATURES = 50000
RANDOM_STATE = 42
TEST_SIZE = 0.20
SUBSET_SIZE = 100000

# =========================
# IMPORT TEAM CODE
# =========================
sys.path.append(os.path.join(BASE_DIR, "team_code"))
import inject_extreme as inject
import clean

# =========================
# UTILITIES
# =========================
def clear_memory():
    gc.collect()

def ensure_dirs():
    for sub in ["baseline", "corrupted", "cleaned", "results", "splits"]:
        os.makedirs(os.path.join(FIXED_DIR, sub), exist_ok=True)

def prepare_baseline_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["text"] = df["text"].astype(str)
    df["label"] = pd.to_numeric(df["label"], errors="coerce")
    df = df.dropna(subset=["label", "text"])
    df["label"] = df["label"].astype(int)
    df = df[df["text"] != "nan"]
    df = df[df["text"].str.len() > 0]
    df = df.reset_index(drop=True)
    df["row_id"] = df.index
    return df

def valid_row_mask(df: pd.DataFrame) -> pd.Series:
    text = df["text"].astype(str)
    label = pd.to_numeric(df["label"], errors="coerce")
    mask = label.notna() & text.notna()
    mask &= (text != "nan") & (text.str.len() > 0)
    mask &= label.apply(lambda x: float(x).is_integer())
    return mask

def build_model():
    return Pipeline([
        ("tfidf", TfidfVectorizer(max_features=TFIDF_MAX_FEATURES, stop_words="english")),
        ("clf", LogisticRegression(max_iter=1000))
    ])

def compute_metrics(y_true, y_pred):
    return {
        "accuracy": float(accuracy_score(y_true, y_pred)),
        "precision": float(precision_score(y_true, y_pred, average="weighted", zero_division=0)),
        "recall": float(recall_score(y_true, y_pred, average="weighted", zero_division=0)),
        "f1": float(f1_score(y_true, y_pred, average="weighted", zero_division=0)),
    }

def train_and_evaluate(df_train, df_test, allow_invalid=False):
    if allow_invalid:
        train_valid = df_train.copy()
        test_valid = df_test.copy()
        train_valid['text'] = train_valid['text'].fillna('')
        test_valid['text'] = test_valid['text'].fillna('')
        train_valid['label'] = pd.to_numeric(train_valid['label'], errors='coerce').fillna(3).astype(int)
        test_valid['label'] = pd.to_numeric(test_valid['label'], errors='coerce').fillna(3).astype(int)
    else:
        train_mask = valid_row_mask(df_train)
        test_mask = valid_row_mask(df_test)
        train_valid = df_train[train_mask].copy()
        test_valid = df_test[test_mask].copy()

    X_train = train_valid["text"].astype(str).values
    y_train = pd.to_numeric(train_valid["label"], errors="coerce").astype(int).values
    X_test = test_valid["text"].astype(str).values
    y_test = pd.to_numeric(test_valid["label"], errors="coerce").astype(int).values

    model = build_model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    metrics = compute_metrics(y_test, y_pred)
    stats = {
        "train_total": len(df_train),
        "train_valid": len(train_valid),
        "test_total": len(df_test),
        "test_valid": len(test_valid),
    }

    del model, X_train, y_train, X_test, y_test, train_valid, test_valid
    clear_memory()
    return metrics, stats

# =========================
# MAIN
# =========================
print("="*80)
print("BATCH 2 EVALUATION - Remaining 6 Corruptions")
print("="*80)

ensure_dirs()

# Load baseline
print("\n📊 Loading baseline...")
baseline_path = os.path.join(BASE_DIR, "data/baseline/amazon_reviews_1M.csv")
df_raw = pd.read_csv(baseline_path)
print(f"   Loaded: {len(df_raw):,} rows")

if len(df_raw) > SUBSET_SIZE:
    df_raw = df_raw.sample(n=SUBSET_SIZE, random_state=RANDOM_STATE)
    print(f"   Sampled: {SUBSET_SIZE:,} rows")

df_baseline = prepare_baseline_data(df_raw)
del df_raw
clear_memory()
print(f"   Cleaned: {len(df_baseline):,} rows")

# Split
df_train_clean, df_test_clean = train_test_split(
    df_baseline, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=df_baseline["label"]
)
train_ids = set(df_train_clean["row_id"].tolist())
test_ids = set(df_test_clean["row_id"].tolist())
print(f"   Split: {len(df_train_clean):,} train / {len(df_test_clean):,} test")

# Baseline eval
print("\n📈 Baseline evaluation...")
metrics_base, stats_base = train_and_evaluate(df_train_clean, df_test_clean, allow_invalid=False)
print(f"   Acc: {metrics_base['accuracy']:.4f} | Prec: {metrics_base['precision']:.4f} | Rec: {metrics_base['recall']:.4f} | F1: {metrics_base['f1']:.4f}")

all_results = []
def record_result(exp_id, stage, metrics, stats):
    row = {"experiment": exp_id, "stage": stage, **stats, **metrics}
    all_results.append(row)

record_result("BASELINE", "CLEAN", metrics_base, stats_base)

# Experiments
experiments = [
    ("05_swapped_labels_manual", inject.apply_swapped_labels_manual, {}),
    ("06_noise_injection", inject.apply_noise_injection, {}),
    ("07_truncation", inject.apply_truncation, {}),
    ("08_combined_jenga", inject.apply_combined_jenga, {}),
    ("09_combined_manual", inject.apply_combined_manual, {}),
    ("10_doomsday", inject.apply_doomsday, {}),
]

for idx, (exp_id, corrupt_func, kwargs) in enumerate(experiments, start=1):
    print(f"\n[{idx}/6] {exp_id}")
    print("-" * 80)

    # Corrupt
    print("   🔴 Corrupting...")
    df_corrupt = corrupt_func(df_baseline.copy(), **kwargs)
    df_corrupt.to_csv(os.path.join(FIXED_DIR, f"corrupted/{exp_id}.csv"), index=False)

    # Split
    df_train_corrupt = df_corrupt[df_corrupt["row_id"].isin(train_ids)].copy()
    df_test_corrupt = df_corrupt[df_corrupt["row_id"].isin(test_ids)].copy()

    # Eval corrupted
    print("   🔴 Training on CORRUPTED...")
    metrics_corrupt, stats_corrupt = train_and_evaluate(df_train_corrupt, df_test_corrupt, allow_invalid=True)
    record_result(exp_id, "CORRUPTED", metrics_corrupt, stats_corrupt)
    print(f"      Acc: {metrics_corrupt['accuracy']:.4f} | Prec: {metrics_corrupt['precision']:.4f} | Rec: {metrics_corrupt['recall']:.4f} | F1: {metrics_corrupt['f1']:.4f}")

    del df_train_corrupt, df_test_corrupt
    clear_memory()

    # Clean
    print("   🟢 Cleaning...")
    df_cleaned = clean.clean_all(df_corrupt)
    df_cleaned.to_csv(os.path.join(FIXED_DIR, f"cleaned/{exp_id}_cleaned.csv"), index=False)
    del df_corrupt
    clear_memory()

    # Split
    df_train_cleaned = df_cleaned[df_cleaned["row_id"].isin(train_ids)].copy()
    df_test_cleaned = df_cleaned[df_cleaned["row_id"].isin(test_ids)].copy()

    # Eval cleaned
    print("   🟢 Training on CLEANED...")
    metrics_cleaned, stats_cleaned = train_and_evaluate(df_train_cleaned, df_test_cleaned, allow_invalid=False)
    record_result(exp_id, "CLEANED", metrics_cleaned, stats_cleaned)
    print(f"      Acc: {metrics_cleaned['accuracy']:.4f} | Prec: {metrics_cleaned['precision']:.4f} | Rec: {metrics_cleaned['recall']:.4f} | F1: {metrics_cleaned['f1']:.4f}")

    # Recovery
    acc_drop = metrics_base['accuracy'] - metrics_corrupt['accuracy']
    acc_recovery = metrics_cleaned['accuracy'] - metrics_corrupt['accuracy']
    recovery_pct = (acc_recovery / acc_drop * 100) if acc_drop != 0 else 0
    print(f"      📊 Recovery: {recovery_pct:.1f}%")

    del df_train_cleaned, df_test_cleaned, df_cleaned
    clear_memory()

# Save results
results_path = os.path.join(FIXED_DIR, "results/batch2_results.csv")
pd.DataFrame(all_results).to_csv(results_path, index=False)

print("\n" + "="*80)
print("BATCH 2 COMPLETE!")
print(f"✅ Results: {results_path}")
print("="*80)

### Part 2: Statitical Tests

In [ ]:
"""
Statistical Analysis - CORRECTED
Uses absolute improvements and proper interpretation
"""

import pandas as pd
import numpy as np
import os

BASE_DIR = "/content/drive/MyDrive/data_preparation_project_2026"

print("="*80)
print("STATISTICAL ANALYSIS - CORRECTED")
print("="*80)

# Load results
batch1 = pd.read_csv(os.path.join(BASE_DIR, "fixed_batch1/results/batch1_results.csv"))
batch2 = pd.read_csv(os.path.join(BASE_DIR, "fixed_batch2/results/batch2_results.csv"))
df_all = pd.concat([batch1, batch2], ignore_index=True)

print(f"\n✅ Loaded {len(df_all)} result rows")

# Get baseline
baseline_acc = df_all[df_all['stage'] == 'CLEAN']['accuracy'].values[0]

# ============================================
# ANALYSIS
# ============================================
stats_results = []

experiments = df_all['experiment'].unique()
experiments = [e for e in experiments if e != 'BASELINE']

for exp in experiments:
    exp_data = df_all[df_all['experiment'] == exp]

    if len(exp_data) < 2:
        continue

    corrupted = exp_data[exp_data['stage'] == 'CORRUPTED']
    cleaned = exp_data[exp_data['stage'] == 'CLEANED']

    if len(corrupted) == 0 or len(cleaned) == 0:
        continue

    acc_corrupt = corrupted['accuracy'].values[0]
    acc_clean = cleaned['accuracy'].values[0]

    # Calculate metrics
    damage = baseline_acc - acc_corrupt
    recovery_abs = acc_clean - acc_corrupt
    recovery_pct = (recovery_abs / damage * 100) if damage != 0 else 0

    # Interpret damage
    def interpret_damage(d):
        abs_d = abs(d)
        if abs_d < 0.03:
            return "Minimal"
        elif abs_d < 0.08:
            return "Moderate"
        elif abs_d < 0.15:
            return "Severe"
        else:
            return "Critical"

    # Interpret recovery
    def interpret_recovery(r):
        if r < 0:
            return "Negative (Cleaning Hurt)"
        elif r < 0.01:
            return "Negligible"
        elif r < 0.05:
            return "Small"
        elif r < 0.10:
            return "Moderate"
        else:
            return "Large"

    # Statistical significance (using sample size and improvement)
    # For n=20k test samples, improvements >1% are typically significant
    n_test = 19997  # Your test size

    # Standard error of accuracy difference
    se = np.sqrt((acc_corrupt * (1 - acc_corrupt) + acc_clean * (1 - acc_clean)) / n_test)

    # Z-score
    if se > 0:
        z_score = recovery_abs / se
        # Two-tailed p-value approximation
        from scipy.stats import norm
        p_value = 2 * (1 - norm.cdf(abs(z_score)))
    else:
        p_value = 1.0

    sig = "***" if p_value < 0.001 else ("**" if p_value < 0.01 else ("*" if p_value < 0.05 else "ns"))

    stats_results.append({
        'Experiment': exp.replace('_', ' ').title(),
        'Baseline': f"{baseline_acc:.3f}",
        'Corrupted': f"{acc_corrupt:.3f}",
        'Cleaned': f"{acc_clean:.3f}",
        'Damage': f"{damage:.3f}",
        'Damage Level': interpret_damage(damage),
        'Recovery (Δ)': f"{recovery_abs:+.3f}",
        'Recovery (%)': f"{recovery_pct:.1f}%",
        'Recovery Level': interpret_recovery(recovery_abs),
        'p-value': f"{p_value:.4f}",
        'Significant': sig,
    })

df_stats = pd.DataFrame(stats_results)

print("\n" + "="*80)
print("CORRUPTION IMPACT & RECOVERY ANALYSIS")
print("="*80)
print("\n" + df_stats.to_string(index=False))

# Save
stats_path = os.path.join(BASE_DIR, "results/statistical_analysis_corrected.csv")
os.makedirs(os.path.join(BASE_DIR, "results"), exist_ok=True)
df_stats.to_csv(stats_path, index=False)
print(f"\n✅ Saved: {stats_path}")

# ============================================
# SUMMARY
# ============================================
print("\n" + "="*80)
print("SUMMARY STATISTICS")
print("="*80)

print(f"\n📊 Corruption Damage Levels:")
for level in ['Minimal', 'Moderate', 'Severe', 'Critical']:
    count = len(df_stats[df_stats['Damage Level'] == level])
    print(f"   {level:12s}: {count}/{len(df_stats)}")

print(f"\n📊 Recovery Effectiveness:")
for level in ['Negative (Cleaning Hurt)', 'Negligible', 'Small', 'Moderate', 'Large']:
    count = len(df_stats[df_stats['Recovery Level'] == level])
    print(f"   {level:30s}: {count}/{len(df_stats)}")

print(f"\n📊 Statistical Significance:")
for sig in ['***', '**', '*', 'ns']:
    count = len(df_stats[df_stats['Significant'] == sig])
    label = "p<0.001" if sig == '***' else ("p<0.01" if sig == '**' else ("p<0.05" if sig == '*' else "p≥0.05"))
    print(f"   {sig:4s} ({label:7s}): {count}/{len(df_stats)}")

# ============================================
# KEY FINDINGS
# ============================================
print("\n" + "="*80)
print("KEY FINDINGS")
print("="*80)

# Most damaging
most_damage_idx = df_stats['Damage'].str.replace('-', '').astype(float).idxmax()
most_damage = df_stats.iloc[most_damage_idx]
print(f"\n💥 Most Damaging Corruption:")
print(f"   {most_damage['Experiment']}")
print(f"   {most_damage['Baseline']} → {most_damage['Corrupted']} (Δ = {most_damage['Damage']})")
print(f"   Level: {most_damage['Damage Level']}")

# Best recovery
best_recovery_idx = df_stats['Recovery (Δ)'].str.replace('+', '').astype(float).idxmax()
best_recovery = df_stats.iloc[best_recovery_idx]
print(f"\n🎯 Best Recovery:")
print(f"   {best_recovery['Experiment']}")
print(f"   Improvement: {best_recovery['Recovery (Δ)']} ({best_recovery['Recovery (%)']})")
print(f"   Level: {best_recovery['Recovery Level']}")
print(f"   Significance: {best_recovery['Significant']} (p={best_recovery['p-value']})")

# Worst recovery (most negative)
worst_recovery_idx = df_stats['Recovery (Δ)'].str.replace('+', '').astype(float).idxmin()
worst_recovery = df_stats.iloc[worst_recovery_idx]
print(f"\n❌ Worst Recovery (Cleaning Hurt):")
print(f"   {worst_recovery['Experiment']}")
print(f"   Change: {worst_recovery['Recovery (Δ)']} ({worst_recovery['Recovery (%)']})")
print(f"   Level: {worst_recovery['Recovery Level']}")

# Count significant improvements
sig_count = len(df_stats[df_stats['Significant'].isin(['*', '**', '***'])])
print(f"\n📊 Overall: {sig_count}/{len(df_stats)} corruptions showed statistically significant recovery")

print("\n" + "="*80)
print("✅ ANALYSIS COMPLETE")
print("="*80)

print("\n📖 Interpretation Guide:")
print("\n   Damage Levels (absolute accuracy drop):")
print("   • Minimal:   < 3%")
print("   • Moderate:  3-8%")
print("   • Severe:    8-15%")
print("   • Critical:  > 15%")
print("\n   Recovery Levels (absolute accuracy gain from cleaning):")
print("   • Negative:  Cleaning made it worse")
print("   • Negligible: < 1% improvement")
print("   • Small:     1-5% improvement")
print("   • Moderate:  5-10% improvement")
print("   • Large:     > 10% improvement")
print("\n   Significance:")
print("   • ***: p < 0.001 (highly significant)")
print("   • **:  p < 0.01  (very significant)")
print("   • *:   p < 0.05  (significant)")
print("   • ns:  p ≥ 0.05  (not significant)")

# TFDV

Change Runtime on Collab!

In [ ]:
!pip install --upgrade pip

!pip install \
  tensorflow==2.15.1 \
  tensorflow-metadata==1.15.0 \
  tensorflow-data-validation==1.15.1


In [ ]:
import sys
import tensorflow as tf
import tensorflow_data_validation as tfdv

print(" Python:", sys.version)
print(" TensorFlow:", tf.__version__)
print(" TFDV:", tfdv.__version__)
print("\n TFDV successfully installed!")

In [ ]:
import os

BASE_DIR = "/content/drive/MyDrive/data_preparation_project_2026"

print("CHECKING YOUR ACTUAL STRUCTURE")
print("="*60)

# Check what exists
for batch in ['fixed_batch1', 'fixed_batch2']:
    print(f"\n{batch}:")
    for sub in ['baseline', 'corrupted', 'cleaned', 'results']:
        path = os.path.join(BASE_DIR, batch, sub)
        if os.path.exists(path):
            files = os.listdir(path)
            print(f"  ✅ {sub}/ ({len(files)} files)")
        else:
            print(f"  ❌ {sub}/ (missing)")

In [ ]:
"""
PART 2: TensorFlow Data Validation (TFDV)
Detect corruption at the data level before model training
"""

import pandas as pd
import numpy as np
import os
import sys
import tensorflow_data_validation as tfdv
from google.protobuf.json_format import MessageToDict

BASE_DIR = "/content/drive/MyDrive/data_preparation_project_2026"

print("="*80)
print("PART 2: TENSORFLOW DATA VALIDATION (TFDV)")
print("="*80)
print("\nGoal: Detect corruption BEFORE training models")
print("Show which corruptions can be caught via data validation\n")

# ============================================
# 1. LOAD BASELINE & GENERATE SCHEMA
# ============================================
print("\n" + "="*80)
print("STEP 1: Generate Schema from Clean Baseline")
print("="*80)

# Try multiple possible baseline locations
baseline_paths = [
    os.path.join(BASE_DIR, "fixed/baseline/amazon_clean_with_rowid.csv"),
    os.path.join(BASE_DIR, "fixed_batch1/baseline/amazon_clean_with_rowid.csv"),
    os.path.join(BASE_DIR, "data/baseline/amazon_reviews_1M.csv"),
]

df_baseline = None
baseline_path = None

for path in baseline_paths:
    if os.path.exists(path):
        baseline_path = path
        df_baseline = pd.read_csv(path)
        break

if df_baseline is None:
    print("❌ ERROR: Could not find baseline file in any expected location:")
    for path in baseline_paths:
        print(f"   - {path}")
    print("\nPlease check your file paths!")
    sys.exit(1)

print(f"✅ Loaded baseline from: {baseline_path}")
print(f"   {len(df_baseline):,} rows")
print(f"   Columns: {list(df_baseline.columns)}")

# Generate statistics from baseline
print("\n📊 Generating baseline statistics...")
baseline_stats = tfdv.generate_statistics_from_dataframe(df_baseline[['text', 'label']])

# Infer schema
print("📋 Inferring schema from baseline...")
schema = tfdv.infer_schema(statistics=baseline_stats)

print("\n✅ Schema inferred:")
print(f"   Features: {[f.name for f in schema.feature]}")

# ============================================
# 2. VALIDATE CORRUPTED DATASETS
# ============================================
print("\n" + "="*80)
print("STEP 2: Validate Corrupted Datasets Against Schema")
print("="*80)

# All corrupted datasets - try multiple directories
corrupted_files = [
    ('01_missing_values', ['fixed_batch1', 'fixed']),
    ('02_broken_chars', ['fixed_batch1', 'fixed']),
    ('03_swapped_text', ['fixed_batch1', 'fixed']),
    ('04_missing_labels', ['fixed_batch1', 'fixed']),
    ('05_swapped_labels_manual', ['fixed_batch2', 'fixed']),
    ('06_noise_injection', ['fixed_batch2', 'fixed']),
    ('07_truncation', ['fixed_batch2', 'fixed']),
    ('08_combined_jenga', ['fixed_batch2', 'fixed']),
    ('09_combined_manual', ['fixed_batch2', 'fixed']),
    ('10_doomsday', ['fixed_batch2', 'fixed']),
]

tfdv_results = []

for exp_id, batch_dirs in corrupted_files:
    print(f"\n{'='*60}")
    print(f"Validating: {exp_id}")
    print(f"{'='*60}")

    try:
        # Try multiple possible paths
        df_corrupt = None
        corrupt_path = None

        for batch_dir in batch_dirs:
            path = os.path.join(BASE_DIR, batch_dir, f"corrupted/{exp_id}.csv")
            if os.path.exists(path):
                corrupt_path = path
                df_corrupt = pd.read_csv(path)
                break

        if df_corrupt is None:
            print(f"   ⚠️  File not found, skipping...")
            continue

        print(f"   Loaded: {len(df_corrupt):,} rows")

        # Generate statistics
        print("   📊 Generating statistics...")
        corrupt_stats = tfdv.generate_statistics_from_dataframe(df_corrupt[['text', 'label']])

        # Validate against schema
        print("   🔍 Validating against schema...")
        anomalies = tfdv.validate_statistics(statistics=corrupt_stats, schema=schema)

        # Parse anomalies
        anomaly_info = MessageToDict(anomalies)

        # Count anomalies
        n_anomalies = len(anomaly_info.get('anomalyInfo', {}))

        # Extract specific anomaly types
        anomaly_types = []
        missing_features = 0
        high_missing = 0
        unexpected_values = 0

        for feature_name, anomaly_data in anomaly_info.get('anomalyInfo', {}).items():
            reason = anomaly_data.get('shortDescription', '')
            anomaly_types.append(f"{feature_name}: {reason}")

            if 'missing' in reason.lower():
                missing_features += 1
                # Check severity
                if 'column dropped' in reason.lower() or 'high' in reason.lower():
                    high_missing += 1

            if 'unexpected' in reason.lower() or 'out of bounds' in reason.lower():
                unexpected_values += 1

        # Calculate drift (compare distributions)
        print("   📉 Calculating distribution drift...")

        # For text: check completeness
        text_complete_baseline = df_baseline['text'].notna().sum() / len(df_baseline)
        text_complete_corrupt = df_corrupt['text'].notna().sum() / len(df_corrupt)
        text_drift = abs(text_complete_baseline - text_complete_corrupt)

        # For label: check completeness + distribution
        label_complete_baseline = df_baseline['label'].notna().sum() / len(df_baseline)
        label_complete_corrupt = df_corrupt['label'].notna().sum() / len(df_corrupt)
        label_drift = abs(label_complete_baseline - label_complete_corrupt)

        # Distribution shift (KS test would be here, simplified to completeness for now)
        overall_drift = (text_drift + label_drift) / 2

        # Detectability score (can TFDV catch this corruption?)
        if n_anomalies > 0 or overall_drift > 0.05:
            detectability = "High"
        elif overall_drift > 0.01:
            detectability = "Medium"
        else:
            detectability = "Low"

        # Print results
        print(f"\n   ✅ TFDV Results:")
        print(f"      Anomalies detected: {n_anomalies}")
        print(f"      Missing features: {missing_features}")
        print(f"      Unexpected values: {unexpected_values}")
        print(f"      Text completeness: {text_complete_corrupt:.1%} (Δ = {text_drift:.1%})")
        print(f"      Label completeness: {label_complete_corrupt:.1%} (Δ = {label_drift:.1%})")
        print(f"      Overall drift: {overall_drift:.1%}")
        print(f"      Detectability: {detectability}")

        # Store results
        tfdv_results.append({
            'Experiment': exp_id.replace('_', ' ').title(),
            'Anomalies': n_anomalies,
            'Missing Features': missing_features,
            'Unexpected Values': unexpected_values,
            'Text Completeness': f"{text_complete_corrupt:.1%}",
            'Text Drift': f"{text_drift:.1%}",
            'Label Completeness': f"{label_complete_corrupt:.1%}",
            'Label Drift': f"{label_drift:.1%}",
            'Overall Drift': f"{overall_drift:.1%}",
            'Detectability': detectability,
            'Anomaly Details': '; '.join(anomaly_types[:3]) if anomaly_types else 'None'
        })

    except Exception as e:
        print(f"   ❌ Error: {e}")
        tfdv_results.append({
            'Experiment': exp_id.replace('_', ' ').title(),
            'Anomalies': 'Error',
            'Missing Features': 'Error',
            'Unexpected Values': 'Error',
            'Text Completeness': 'Error',
            'Text Drift': 'Error',
            'Label Completeness': 'Error',
            'Label Drift': 'Error',
            'Overall Drift': 'Error',
            'Detectability': 'Error',
            'Anomaly Details': str(e)
        })

# ============================================
# 3. SAVE RESULTS
# ============================================
print("\n" + "="*80)
print("TFDV RESULTS SUMMARY")
print("="*80)

df_tfdv = pd.DataFrame(tfdv_results)
print("\n" + df_tfdv.to_string(index=False))

# Save
tfdv_path = os.path.join(BASE_DIR, "results/PART2_TFDV_Analysis.csv")
os.makedirs(os.path.join(BASE_DIR, "results"), exist_ok=True)
df_tfdv.to_csv(tfdv_path, index=False)

print(f"\n✅ Saved: {tfdv_path}")

# ============================================
# 4. SUMMARY STATISTICS
# ============================================
print("\n" + "="*80)
print("DETECTABILITY ANALYSIS")
print("="*80)

# Filter out errors
df_tfdv_valid = df_tfdv[df_tfdv['Detectability'] != 'Error']

print(f"\n📊 Corruption Detectability via TFDV:")
for level in ['High', 'Medium', 'Low']:
    count = len(df_tfdv_valid[df_tfdv_valid['Detectability'] == level])
    total = len(df_tfdv_valid)
    pct = count / total * 100 if total > 0 else 0
    print(f"   {level:8s}: {count}/{total} ({pct:.0f}%)")

# Average drift by detectability
if len(df_tfdv_valid) > 0:
    print(f"\n📊 Average Drift by Detectability:")
    for level in ['High', 'Medium', 'Low']:
        subset = df_tfdv_valid[df_tfdv_valid['Detectability'] == level]
        if len(subset) > 0:
            # Convert drift strings to float
            drifts = subset['Overall Drift'].str.rstrip('%').astype(float)
            avg_drift = drifts.mean()
            print(f"   {level:8s}: {avg_drift:.1f}% average drift")

# Anomaly detection rate
if len(df_tfdv_valid) > 0:
    detected = len(df_tfdv_valid[df_tfdv_valid['Anomalies'] > 0])
    total = len(df_tfdv_valid)
    print(f"\n📊 Detection Rate:")
    print(f"   {detected}/{total} corruptions generated anomalies ({detected/total*100:.0f}%)")

print("\n" + "="*80)
print("✅ PART 2 COMPLETE - TFDV ANALYSIS DONE!")
print("="*80)

print("\n🔑 Key Insight:")
print("   TFDV can detect structural corruptions (missing values, labels)")
print("   but struggles with semantic corruptions (swapped labels, noise)")
print("   This validates the need for model-based evaluation (Part 1)")

In [ ]:
"""
PART 2: TENSORFLOW DATA VALIDATION (TFDV) - COMPREHENSIVE
Complete end-to-end data validation analysis
"""

import pandas as pd
import numpy as np
import os
import sys
import tensorflow_data_validation as tfdv
from google.protobuf.json_format import MessageToDict
from scipy.stats import ks_2samp, entropy
from scipy.spatial.distance import jensenshannon

BASE_DIR = "/content/drive/MyDrive/data_preparation_project_2026"

print("="*80)
print("PART 2: TENSORFLOW DATA VALIDATION (TFDV) - COMPREHENSIVE")
print("="*80)
print("\nEnd-to-end data quality assessment:")
print("1. Schema validation & anomaly detection")
print("2. Distribution drift & skew analysis")
print("3. Feature-level statistics")
print("4. Corruption detectability scoring")
print()

# ============================================
# 1. LOAD BASELINE & GENERATE SCHEMA
# ============================================
print("="*80)
print("STEP 1: Generate Schema from Clean Baseline")
print("="*80)

baseline_paths = [
    os.path.join(BASE_DIR, "fixed/baseline/amazon_clean_with_rowid.csv"),
    os.path.join(BASE_DIR, "data/baseline/amazon_reviews_1M.csv"),
]

baseline_path = None
for path in baseline_paths:
    if os.path.exists(path):
        baseline_path = path
        break

if baseline_path is None:
    print("❌ ERROR: Could not find baseline file")
    sys.exit(1)

print(f"✅ Loaded baseline from: {baseline_path}")
df_baseline = pd.read_csv(baseline_path)

# Clean if needed
if 'amazon_reviews_1M.csv' in baseline_path:
    print("   🧹 Cleaning raw baseline...")
    df_baseline["text"] = df_baseline["text"].astype(str)
    df_baseline["label"] = pd.to_numeric(df_baseline["label"], errors="coerce")
    df_baseline = df_baseline.dropna(subset=["label", "text"])
    df_baseline["label"] = df_baseline["label"].astype(int)
    df_baseline = df_baseline[df_baseline["text"] != "nan"]
    df_baseline = df_baseline[df_baseline["text"].str.len() > 0]
    if len(df_baseline) > 100000:
        df_baseline = df_baseline.sample(n=100000, random_state=42)

print(f"   {len(df_baseline):,} rows")

# Generate baseline statistics
print("\n📊 Generating baseline statistics...")
baseline_stats = tfdv.generate_statistics_from_dataframe(df_baseline[['text', 'label']])

# Infer schema
print("📋 Inferring schema...")
schema = tfdv.infer_schema(statistics=baseline_stats)

# Get baseline label distribution
baseline_label_dist = df_baseline['label'].value_counts(normalize=True).sort_index()
print(f"\n📊 Baseline Label Distribution:")
for label, pct in baseline_label_dist.items():
    print(f"   Label {label}: {pct:.1%}")

print(f"\n✅ Schema inferred with {len(schema.feature)} features")

# ============================================
# 2. COMPREHENSIVE VALIDATION
# ============================================
print("\n" + "="*80)
print("STEP 2: Comprehensive Corruption Analysis")
print("="*80)

corrupted_files = [
    ('01_missing_values', ['fixed', 'fixed_batch1']),
    ('02_broken_chars', ['fixed', 'fixed_batch1']),
    ('03_swapped_text', ['fixed', 'fixed_batch1']),
    ('04_missing_labels', ['fixed', 'fixed_batch1']),
    ('05_swapped_labels_manual', ['fixed', 'fixed_batch2']),
    ('06_noise_injection', ['fixed', 'fixed_batch2']),
    ('07_truncation', ['fixed', 'fixed_batch2']),
    ('08_combined_jenga', ['fixed', 'fixed_batch2']),
    ('09_combined_manual', ['fixed', 'fixed_batch2']),
    ('10_doomsday', ['fixed', 'fixed_batch2']),
]

tfdv_results = []

for exp_id, batch_dirs in corrupted_files:
    print(f"\n{'='*80}")
    print(f"Analyzing: {exp_id}")
    print(f"{'='*80}")

    try:
        # Find corrupted file
        df_corrupt = None
        for batch_dir in batch_dirs:
            path = os.path.join(BASE_DIR, batch_dir, f"corrupted/{exp_id}.csv")
            if os.path.exists(path):
                df_corrupt = pd.read_csv(path)
                break

        if df_corrupt is None:
            print(f"   ⚠️  File not found, skipping...")
            continue

        print(f"   Loaded: {len(df_corrupt):,} rows")

        # ==================
        # A. SCHEMA VALIDATION
        # ==================
        print("   📊 A. Schema Validation...")
        corrupt_stats = tfdv.generate_statistics_from_dataframe(df_corrupt[['text', 'label']])
        anomalies = tfdv.validate_statistics(statistics=corrupt_stats, schema=schema)

        anomaly_info = MessageToDict(anomalies)
        n_anomalies = len(anomaly_info.get('anomalyInfo', {}))

        anomaly_types = []
        for feature_name, anomaly_data in anomaly_info.get('anomalyInfo', {}).items():
            reason = anomaly_data.get('shortDescription', '')
            anomaly_types.append(f"{feature_name}: {reason}")

        # ==================
        # B. COMPLETENESS DRIFT
        # ==================
        print("   📊 B. Completeness Analysis...")
        text_complete_base = df_baseline['text'].notna().sum() / len(df_baseline)
        text_complete_corrupt = df_corrupt['text'].notna().sum() / len(df_corrupt)
        text_drift = abs(text_complete_base - text_complete_corrupt)

        label_complete_base = df_baseline['label'].notna().sum() / len(df_baseline)
        label_complete_corrupt = df_corrupt['label'].notna().sum() / len(df_corrupt)
        label_drift = abs(label_complete_base - label_complete_corrupt)

        # ==================
        # C. DISTRIBUTION SKEW
        # ==================
        print("   📊 C. Distribution Skew Analysis...")

        # Label distribution comparison
        corrupt_label_dist = df_corrupt['label'].value_counts(normalize=True).sort_index()

        # Align distributions (fill missing labels with 0)
        all_labels = sorted(set(baseline_label_dist.index) | set(corrupt_label_dist.index))
        base_aligned = [baseline_label_dist.get(l, 0) for l in all_labels]
        corrupt_aligned = [corrupt_label_dist.get(l, 0) for l in all_labels]

        # L1 distance (total variation)
        l1_distance = sum(abs(b - c) for b, c in zip(base_aligned, corrupt_aligned)) / 2

        # Jensen-Shannon divergence (symmetric KL divergence)
        # Add small epsilon to avoid log(0)
        base_smooth = [p + 1e-10 for p in base_aligned]
        corrupt_smooth = [p + 1e-10 for p in corrupt_aligned]
        js_divergence = jensenshannon(base_smooth, corrupt_smooth)

        # Kolmogorov-Smirnov test (for numeric labels)
        ks_statistic, ks_pvalue = ks_2samp(
            df_baseline['label'].dropna().values,
            df_corrupt['label'].dropna().values
        )

        # ==================
        # D. FEATURE STATISTICS
        # ==================
        print("   📊 D. Feature-Level Statistics...")

        # Text length statistics
        base_text_lengths = df_baseline['text'].str.len()
        corrupt_text_lengths = df_corrupt['text'].str.len()

        avg_length_base = base_text_lengths.mean()
        avg_length_corrupt = corrupt_text_lengths.mean()
        length_change = ((avg_length_corrupt - avg_length_base) / avg_length_base * 100) if avg_length_base > 0 else 0

        # ==================
        # E. DETECTABILITY SCORING
        # ==================
        print("   📊 E. Detectability Scoring...")

        # Composite detectability score
        completeness_score = (text_drift + label_drift) / 2
        distribution_score = l1_distance
        anomaly_score = min(n_anomalies / 2, 1.0)  # Normalize to 0-1

        # Weighted composite (40% completeness, 40% distribution, 20% anomalies)
        detectability_score = (
            0.4 * completeness_score +
            0.4 * distribution_score +
            0.2 * anomaly_score
        )

        if detectability_score > 0.2:
            detectability = "High"
        elif detectability_score > 0.05:
            detectability = "Medium"
        else:
            detectability = "Low"

        # ==================
        # F. PRINT RESULTS
        # ==================
        print(f"\n   ✅ VALIDATION RESULTS:")
        print(f"      Schema Anomalies: {n_anomalies}")
        print(f"      Text Completeness: {text_complete_corrupt:.1%} (Δ = {text_drift:.1%})")
        print(f"      Label Completeness: {label_complete_corrupt:.1%} (Δ = {label_drift:.1%})")
        print(f"      Label Distribution L1: {l1_distance:.3f}")
        print(f"      Jensen-Shannon Divergence: {js_divergence:.3f}")
        print(f"      KS Statistic: {ks_statistic:.3f} (p={ks_pvalue:.4f})")
        print(f"      Avg Text Length Change: {length_change:+.1f}%")
        print(f"      Detectability Score: {detectability_score:.3f} ({detectability})")

        # Store results
        tfdv_results.append({
            'Experiment': exp_id.replace('_', ' ').title(),
            'Schema Anomalies': n_anomalies,
            'Text Completeness': f"{text_complete_corrupt:.1%}",
            'Text Drift': f"{text_drift:.1%}",
            'Label Completeness': f"{label_complete_corrupt:.1%}",
            'Label Drift': f"{label_drift:.1%}",
            'L1 Distance': f"{l1_distance:.3f}",
            'JS Divergence': f"{js_divergence:.3f}",
            'KS Statistic': f"{ks_statistic:.3f}",
            'KS p-value': f"{ks_pvalue:.4f}",
            'Text Length Change': f"{length_change:+.1f}%",
            'Detectability Score': f"{detectability_score:.3f}",
            'Detectability': detectability,
            'Anomaly Details': '; '.join(anomaly_types[:3]) if anomaly_types else 'None'
        })

    except Exception as e:
        print(f"   ❌ Error: {e}")
        import traceback
        traceback.print_exc()
        continue

# ============================================
# 3. SAVE COMPREHENSIVE RESULTS
# ============================================
print("\n" + "="*80)
print("COMPREHENSIVE TFDV RESULTS")
print("="*80)

df_tfdv = pd.DataFrame(tfdv_results)
print("\n" + df_tfdv.to_string(index=False))

# Save
tfdv_path = os.path.join(BASE_DIR, "results/PART2_TFDV_Comprehensive.csv")
os.makedirs(os.path.join(BASE_DIR, "results"), exist_ok=True)
df_tfdv.to_csv(tfdv_path, index=False)

print(f"\n✅ Saved: {tfdv_path}")

# ============================================
# 4. ANALYSIS SUMMARY
# ============================================
print("\n" + "="*80)
print("ANALYSIS SUMMARY")
print("="*80)

# Detectability distribution
print(f"\n📊 Corruption Detectability:")
for level in ['High', 'Medium', 'Low']:
    count = len(df_tfdv[df_tfdv['Detectability'] == level])
    pct = count / len(df_tfdv) * 100 if len(df_tfdv) > 0 else 0
    print(f"   {level:8s}: {count}/{len(df_tfdv)} ({pct:.0f}%)")

# Anomaly detection rate
detected = len(df_tfdv[df_tfdv['Schema Anomalies'] > 0])
print(f"\n📊 Detection Metrics:")
print(f"   Anomaly Detection Rate: {detected}/{len(df_tfdv)} ({detected/len(df_tfdv)*100:.0f}%)")

# Distribution shift significance
sig_ks = len(df_tfdv[df_tfdv['KS p-value'].astype(float) < 0.05])
print(f"   Significant Distribution Shifts: {sig_ks}/{len(df_tfdv)} ({sig_ks/len(df_tfdv)*100:.0f}%)")

# Average metrics by detectability
print(f"\n📊 Average Drift by Detectability:")
for level in ['High', 'Medium', 'Low']:
    subset = df_tfdv[df_tfdv['Detectability'] == level]
    if len(subset) > 0:
        avg_score = subset['Detectability Score'].astype(float).mean()
        print(f"   {level:8s}: Score = {avg_score:.3f}")

print("\n" + "="*80)
print("✅ PART 2 COMPLETE - COMPREHENSIVE TFDV ANALYSIS!")
print("="*80)

print("\n🔑 Key Findings:")
print(f"   • {detected}/{len(df_tfdv)} corruptions detected via schema anomalies")
print(f"   • {sig_ks}/{len(df_tfdv)} corruptions show significant distribution shifts")
print("   • Structural corruptions (missing values/labels) have high detectability")
print("   • Semantic corruptions (swapped/noise) show low detectability")
print("   • TFDV is necessary but insufficient - model evaluation required")

# ADULT DATASET

In [ ]:
"""
ADULT DATASET EVALUATION - TEAM'S EXACT APPROACH
================================================

Their actual approach (from notebook):
1. Use ONLY "occupation" column as "text"
2. Use TfidfVectorizer + LogisticRegression (same as Amazon!)
3. Apply their inject/clean modules
4. That's it!
"""

import os
import sys
import pandas as pd
import numpy as np
import gc

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIGURATION
# =========================
BASE_DIR = "/content/drive/MyDrive/data_preparation_project_2026"
FIXED_DIR = os.path.join(BASE_DIR, "fixed_adult")
TFIDF_MAX_FEATURES = 5000
RANDOM_STATE = 42
TEST_SIZE = 0.20

# =========================
# IMPORT TEAM CODE
# =========================
sys.path.append(os.path.join(BASE_DIR, "team_code"))
import inject_extreme as inject
import clean

print("="*80)
print("ADULT DATASET EVALUATION - TEAM'S EXACT APPROACH")
print("="*80)

# =========================
# CREATE DIRECTORIES
# =========================
def ensure_dirs():
    for sub in ["baseline", "corrupted", "cleaned", "results"]:
        os.makedirs(os.path.join(FIXED_DIR, sub), exist_ok=True)

ensure_dirs()

# =========================
# LOAD ADULT DATASET
# =========================
print("\n📊 Loading Adult Dataset...")

ADULT_PATH = os.path.join(BASE_DIR, "data/raw/adult.csv")
df_raw = pd.read_csv(ADULT_PATH)
print(f"   Loaded: {len(df_raw):,} rows")

# Remove rows with missing values
df_raw = df_raw.dropna()

# Handle column naming
if 'class' in df_raw.columns:
    target_col = 'class'
elif 'income' in df_raw.columns:
    target_col = 'income'
else:
    raise ValueError("Target column not found!")

# Clean target column
df_raw[target_col] = df_raw[target_col].str.replace('.', '', regex=False).str.strip()

print(f"   After dropna: {len(df_raw):,} rows")

# =========================
# PREPARE IN TEXT/LABEL FORMAT (THEIR EXACT WAY)
# =========================
print("\n🔧 Converting to text/label format (EXACTLY like their notebook)...")

# THEIR EXACT APPROACH: Just use occupation as "text"!
df_baseline = pd.DataFrame()
df_baseline['row_id'] = df_raw.reset_index(drop=True).index
df_baseline['text'] = df_raw['occupation'].astype(str)  # Just occupation!
df_baseline['label'] = df_raw[target_col].map({'<=50K': 1, '>50K': 5})

# Drop any rows where mapping failed
df_baseline = df_baseline.dropna(subset=['label'])
df_baseline['label'] = df_baseline['label'].astype(int)

print(f"   Baseline created: {len(df_baseline):,} rows")
print(f"   text = occupation column")
print(f"   label = income (1 for <=50K, 5 for >50K)")
print(f"\n   Sample:")
print(df_baseline.head(3))

# =========================
# FIXED TRAIN/TEST SPLIT
# =========================
print("\n✂️  Creating fixed train/test split...")

df_train_clean, df_test_clean = train_test_split(
    df_baseline,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=df_baseline['label']
)

train_ids = set(df_train_clean["row_id"].tolist())
test_ids = set(df_test_clean["row_id"].tolist())

print(f"   Train: {len(train_ids):,} samples")
print(f"   Test: {len(test_ids):,} samples")

# =========================
# BUILD MODEL (THEIR EXACT MODEL - TfidfVectorizer + LogisticRegression)
# =========================
print("\n🏗️  Building model (EXACT same as their Amazon/Adult model)...")

def build_model():
    """Their exact model: TfidfVectorizer + LogisticRegression"""
    return Pipeline([
        ("tfidf", TfidfVectorizer(
            max_features=TFIDF_MAX_FEATURES,
            stop_words="english"
        )),
        ("clf", LogisticRegression(max_iter=1000))
    ])

print("   ✅ Model built: TfidfVectorizer + LogisticRegression")

# =========================
# UTILITIES
# =========================
def clear_memory():
    gc.collect()

def valid_row_mask(df):
    """Check if rows are valid for training"""
    text = df["text"].astype(str)
    label = pd.to_numeric(df["label"], errors="coerce")
    mask = label.notna() & text.notna()
    mask &= (text != "nan") & (text.str.len() > 0)
    mask &= label.apply(lambda x: float(x).is_integer() if pd.notna(x) else False)
    return mask

def compute_metrics(y_true, y_pred):
    return {
        "accuracy": float(accuracy_score(y_true, y_pred)),
        "precision": float(precision_score(y_true, y_pred, average="weighted", zero_division=0)),
        "recall": float(recall_score(y_true, y_pred, average="weighted", zero_division=0)),
        "f1": float(f1_score(y_true, y_pred, average="weighted", zero_division=0)),
    }

def train_and_evaluate(df_train, df_test, allow_invalid=False):
    """
    Train and evaluate model
    """
    if allow_invalid:
        # For corrupted data: fill NaN with defaults
        train_valid = df_train.copy()
        test_valid = df_test.copy()
        train_valid['text'] = train_valid['text'].fillna('unknown')
        test_valid['text'] = test_valid['text'].fillna('unknown')
        train_valid['label'] = pd.to_numeric(train_valid['label'], errors='coerce').fillna(3).astype(int)
        test_valid['label'] = pd.to_numeric(test_valid['label'], errors='coerce').fillna(3).astype(int)
    else:
        # For clean data: strict filtering
        train_mask = valid_row_mask(df_train)
        test_mask = valid_row_mask(df_test)
        train_valid = df_train[train_mask].copy()
        test_valid = df_test[test_mask].copy()

    X_train = train_valid["text"].astype(str).values
    y_train = pd.to_numeric(train_valid["label"], errors="coerce").astype(int).values
    X_test = test_valid["text"].astype(str).values
    y_test = pd.to_numeric(test_valid["label"], errors="coerce").astype(int).values

    model = build_model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    metrics = compute_metrics(y_test, y_pred)
    stats = {
        "train_total": len(df_train),
        "train_valid": len(train_valid),
        "test_total": len(df_test),
        "test_valid": len(test_valid),
    }

    del model, X_train, y_train, X_test, y_test, train_valid, test_valid
    clear_memory()

    return metrics, stats

# =========================
# BASELINE EVALUATION
# =========================
print("\n📈 Baseline evaluation...")

metrics_base, stats_base = train_and_evaluate(df_train_clean, df_test_clean, allow_invalid=False)

print(f"   Acc: {metrics_base['accuracy']:.4f} | "
      f"Prec: {metrics_base['precision']:.4f} | "
      f"Rec: {metrics_base['recall']:.4f} | "
      f"F1: {metrics_base['f1']:.4f}")

all_results = []

def record_result(exp_id, stage, metrics, stats):
    row = {"experiment": exp_id, "stage": stage, **stats, **metrics}
    all_results.append(row)

record_result("BASELINE", "CLEAN", metrics_base, stats_base)

# =========================
# CORRUPTION EXPERIMENTS (THEIR EXACT CORRUPTIONS)
# =========================
print("\n🧪 Running corruption experiments...")

experiments = [
    ("01_missing_values", inject.apply_missing_values, {}),
    ("02_broken_chars", inject.apply_broken_characters, {}),
    ("03_swapped_text", inject.apply_swapped_text, {}),
    ("04_missing_labels", inject.apply_missing_labels, {}),
]

for idx, (exp_id, corrupt_func, kwargs) in enumerate(experiments, start=1):
    print(f"\n[{idx}/{len(experiments)}] {exp_id}")
    print("-" * 80)

    # === CORRUPT ===
    print("   🔴 Corrupting...")
    df_corrupt = corrupt_func(df_baseline.copy(), **kwargs)
    df_corrupt.to_csv(os.path.join(FIXED_DIR, f"corrupted/{exp_id}.csv"), index=False)

    # Split
    df_train_corrupt = df_corrupt[df_corrupt["row_id"].isin(train_ids)].copy()
    df_test_corrupt = df_corrupt[df_corrupt["row_id"].isin(test_ids)].copy()

    # === EVALUATE CORRUPTED ===
    print("   🔴 Training on CORRUPTED...")
    metrics_corrupt, stats_corrupt = train_and_evaluate(df_train_corrupt, df_test_corrupt, allow_invalid=True)
    record_result(exp_id, "CORRUPTED", metrics_corrupt, stats_corrupt)

    print(f"      Acc: {metrics_corrupt['accuracy']:.4f} | "
          f"Prec: {metrics_corrupt['precision']:.4f} | "
          f"Rec: {metrics_corrupt['recall']:.4f} | "
          f"F1: {metrics_corrupt['f1']:.4f}")

    del df_train_corrupt, df_test_corrupt
    clear_memory()

    # === CLEAN ===
    print("   🟢 Cleaning...")
    df_cleaned = clean.clean_all(df_corrupt)
    df_cleaned.to_csv(os.path.join(FIXED_DIR, f"cleaned/{exp_id}_cleaned.csv"), index=False)
    del df_corrupt
    clear_memory()

    # Split
    df_train_cleaned = df_cleaned[df_cleaned["row_id"].isin(train_ids)].copy()
    df_test_cleaned = df_cleaned[df_cleaned["row_id"].isin(test_ids)].copy()

    # === EVALUATE CLEANED ===
    print("   🟢 Training on CLEANED...")
    metrics_cleaned, stats_cleaned = train_and_evaluate(df_train_cleaned, df_test_cleaned, allow_invalid=False)
    record_result(exp_id, "CLEANED", metrics_cleaned, stats_cleaned)

    print(f"      Acc: {metrics_cleaned['accuracy']:.4f} | "
          f"Prec: {metrics_cleaned['precision']:.4f} | "
          f"Rec: {metrics_cleaned['recall']:.4f} | "
          f"F1: {metrics_cleaned['f1']:.4f}")

    # === RECOVERY ===
    acc_drop = metrics_base['accuracy'] - metrics_corrupt['accuracy']
    acc_recovery = metrics_cleaned['accuracy'] - metrics_corrupt['accuracy']
    recovery_pct = (acc_recovery / acc_drop * 100) if acc_drop != 0 else 0

    print(f"      📊 Recovery: {recovery_pct:.1f}%")

    del df_train_cleaned, df_test_cleaned, df_cleaned
    clear_memory()

# =========================
# SAVE RESULTS
# =========================
print("\n" + "="*80)
print("SAVING RESULTS")
print("="*80)

results_path = os.path.join(FIXED_DIR, "results/adult_results.csv")
df_results = pd.DataFrame(all_results)
df_results.to_csv(results_path, index=False)

print(f"\n✅ Results saved: {results_path}")
print(f"\n📊 SUMMARY:")
print(df_results[['experiment', 'stage', 'accuracy', 'f1']].to_string(index=False))

print("\n" + "="*80)
print("✅ ADULT DATASET EVALUATION COMPLETE!")
print("="*80)